In [20]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def load_images(image_dir, num_images=5):
    images = {}
    for i, filename in enumerate(os.listdir(image_dir)):
        if i >= num_images:
            break
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is not None:
            images[filename] = img
    return images

def segment_image(image, k):
    pixel_values = image.reshape((-1, 3))
    pixel_values = np.float32(pixel_values)
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    labels = kmeans.fit_predict(pixel_values)
    segmented_image = kmeans.cluster_centers_[labels]
    segmented_image = segmented_image.reshape(image.shape)
    return np.uint8(segmented_image)

def compare_with_mask(segmented_image, mask):
    # Ensure both images are grayscale
    if len(segmented_image.shape) == 3:
        segmented_image = cv2.cvtColor(segmented_image, cv2.COLOR_BGR2GRAY)
    if len(mask.shape) == 3:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    # Ensure both images have the same shape and type
    if segmented_image.shape != mask.shape:
        raise ValueError("Segmented image and mask must have the same shape")
    
    # Perform elementwise comparison
    return np.sum(segmented_image.astype(np.uint8) == mask.astype(np.uint8)) / segmented_image.size

def plot_results(results):
    fig = make_subplots(rows=1, cols=1)
    fig.add_trace(go.Scatter(x=list(results.keys()), y=list(results.values()), mode='lines+markers', name='Accuracy'))
    fig.update_layout(title='K-means Clustering Accuracy for Different K', xaxis_title='K', yaxis_title='Accuracy')
    fig.show()

image_dir = 'Data/Image'
mask_dir = 'Data/Mask'
images = load_images(image_dir)
masks = load_images(mask_dir)

# Ensure filenames in masks match those in images
masks = {os.path.basename(filename): mask for filename, mask in masks.items()}

k_values = range(2, 11)
results = {}

for k in k_values:
    accuracies = []
    for filename, img in images.items():
        if filename in masks:
            mask = masks[filename]
            segmented_image = segment_image(img, k)
            accuracy = compare_with_mask(segmented_image, mask)
            print(f"Accuracy for {filename} with k={k}: {accuracy}")
            accuracies.append(accuracy)
    if accuracies:
        results[k] = np.mean(accuracies)
    else:
        results[k] = 0  # or any other value you deem appropriate

plot_results(results)
